# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [55]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def get_team_name(full_name):
    full_name = full_name.strip()
    
    # only one word or blank
    if (len(full_name.split()) == 1):
        return full_name
    else:
        return full_name.split()[-1]

    
def clean_nhl():
    df = nhl_df
    
    # ensure that data is from 2018
    df = df[df['year'] == 2018]
    
    # remove asterisks from team name
    df['team'] = df['team'].str.replace('*', '')
    
    # remove the rows with 'Division'
    df = df[~df.team.str.contains('Division')]
    
    # separate out the city with the team name and get the last bit of the team name 
    df['team'] = df['team'].apply(lambda x: get_team_name(x))
    
    # create Win/Loss ratio
    df['W/L Ratio'] = df['W'].astype(int)/(df['W'].astype(int) + df['L'].astype(int))
    
    # set index to team
    df.set_index('team', inplace=True)
    
    return df


    

def clean_cities_nhl():
    df = cities
    
    # remove notes or dashes from team names
    df['NHL'] = df['NHL'].apply(lambda x: re.sub('\[.*\]|—', '', x))
    
    # remove blank NHL team names
    df = df[df['NHL'] != '']
    
    # make manual changes for cities that have multiple teams or more than one word
    df['NHL'] = df['NHL'].replace({'Rangers Islanders Devils': 'Rangers,Islanders,Devils',
                                   'Kings Ducks': 'Kings,Ducks'})
    df['NHL'] = df['NHL'].apply(lambda x: get_team_name(x))
    df['NHL'] = df['NHL'].str.split(",")
    df = df.explode('NHL')
    
    # set index to team
    df.set_index('NHL', inplace=True)
    
    df = df[['Metropolitan area', 'Population (2016 est.)[8]']]
    
    df['Population (2016 est.)[8]'] = df['Population (2016 est.)[8]'].astype(int)
    
    return df




def nhl_correlation(): 
    nhl_df = clean_nhl()
    cities_nhl = clean_cities_nhl()
    
    merged_nhl = pd.merge(nhl_df, cities_nhl, how='left', left_index=True, right_index=True)
    
    population_by_region = cities_nhl.drop_duplicates('Metropolitan area').sort_values('Metropolitan area')['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_nhl.groupby('Metropolitan area').agg({'W/L Ratio': np.nanmean})['W/L Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr, _ = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr


dtype('float64')

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [56]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nba():
    df = nba_df
    
    # ensure data is from 2018
    df = df[df['year'] == 2018]
    
    # remove ()'s from team name
    df['team'] = df['team'].apply(lambda x: re.sub("\(.*\)", '', x))
    
    # remove * from team name
    df['team'] = df['team'].str.replace('*','')
    
    # separate out the city with the team name and get the last bit of the team name 
    df['team'] = df['team'].apply(lambda x: get_team_name(x))
    
    # make W/L% a float
    df['W/L%'] = df['W/L%'].astype(float)
    
    df.set_index('team', inplace=True)
    
    return df



def clean_cities_nba():
    df = cities
    
    # remove notes or dashes from team names
    df['NBA'] = df['NBA'].apply(lambda x: re.sub('\[.*\]|—', '', x))
    
    # remove blank NBA team names
    df = df[df['NBA'] != '']
    
    # make manual changes for cities that have multiple teams or more than one word
    df['NBA'] = df['NBA'].replace({'Knicks Nets': 'Knicks,Nets',
                                  'Lakers Clippers': 'Lakers,Clippers'})
    df['NBA'] = df['NBA'].apply(lambda x: get_team_name(x))
    df['NBA'] = df['NBA'].str.split(",")
    df = df.explode('NBA')
    
    # set index to team
    df.set_index('NBA', inplace=True)
    
    df = df[['Metropolitan area', 'Population (2016 est.)[8]']]
    
    df['Population (2016 est.)[8]'] = df['Population (2016 est.)[8]'].astype(int)
    
    return df
    
    
    

def nba_correlation():
    nba_df = clean_nba()
    cities_nba = clean_cities_nba()
    
    merged_nba = pd.merge(nba_df, cities_nba, how='left', left_index=True, right_index=True)
    
    population_by_region = cities_nba.drop_duplicates('Metropolitan area').sort_values('Metropolitan area')['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_nba.groupby('Metropolitan area').agg({'W/L%': np.nanmean})['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    corr, _ = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr


-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def get_team_name_mlb(full_name):
    full_name = full_name.strip()
    
    # only one word or blank
    if (len(full_name.split()) == 1):
        return full_name
    elif ('White Sox' in full_name) | ('Red Sox' in full_name):
        return ' '.join(full_name.split()[-2:])
    else:
        return full_name.split()[-1]
    

def clean_mlb():
    df = mlb_df
    
    # ensure data is from 2018
    df = df[df['year'] == 2018]
    
    # separate out the city with the team name and get the last bit of the team name 
    df['team'] = df['team'].apply(lambda x: get_team_name_mlb(x))
    
    # make W/L% a float
    df['W-L%'] = df['W-L%'].astype(float)
    
    df.set_index('team', inplace=True)
    
    return df


def clean_cities_mlb():
    df = cities
    
    # remove notes or dashes from team names
    df['MLB'] = df['MLB'].apply(lambda x: re.sub('\[.*\]|—', '', x))
    
    # remove blank MLB team names
    df = df[df['MLB'] != '']
    
    # make manual changes for cities that have multiple teams or more than one word
    df['MLB'] = df['MLB'].replace({'Yankees Mets': 'Yankees,Mets',
                                  'Dodgers Angels': 'Dodgers,Angels',
                                  'Giants Athletics': 'Giants,Athletics',
                                  'Cubs White Sox': 'Cubs,White Sox'})
    df['MLB'] = df['MLB'].apply(lambda x: get_team_name_mlb(x))
    df['MLB'] = df['MLB'].str.split(",")
    df = df.explode('MLB')
    
    # set index to team
    df.set_index('MLB', inplace=True)
    
    df = df[['Metropolitan area', 'Population (2016 est.)[8]']]
    
    df['Population (2016 est.)[8]'] = df['Population (2016 est.)[8]'].astype(int)
    
    return df




def mlb_correlation(): 
    mlb_df = clean_mlb()
    cities_mlb = clean_cities_mlb()
    
    merged_df = pd.merge(mlb_df, cities_mlb, how='left', left_index=True, right_index=True)
    
    population_by_region = cities_mlb.drop_duplicates('Metropolitan area').sort_values('Metropolitan area')['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df.groupby('Metropolitan area').agg({'W-L%': np.nanmean})['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    corr, _ = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nfl():
    df = nfl_df
    
    # ensure 2018 data
    df = df[df['year'] == 2018]
    
    # remove asterisks and pluses from team name
    df['team'] = df['team'].str.replace('[*+]', '', regex=True)
    
    # remove the rows with 'AFC' or 'NFC'
    df = df[~df['team'].str.contains('FC')]
    
    # separate out the city with the team name and get the last bit of the team name 
    df['team'] = df['team'].apply(lambda x: get_team_name(x))
    
    # make W-L% a float
    df['W-L%'] = df['W-L%'].astype(float)
    
    df.set_index('team', inplace=True)
    
    return df


def clean_cities_nfl():
    df = cities
    
    # remove notes or dashes from team names
    df['NFL'] = df['NFL'].apply(lambda x: re.sub('\[.*\]|—', '', x))
    
    df['NFL'] = df['NFL'].str.strip()
    
    # remove blank NFL team names
    df = df[df['NFL'] != '']
    
    # make manual changes for cities that have multiple teams or more than one word
    df['NFL'] = df['NFL'].replace({'Giants Jets': 'Giants,Jets',
                                  'Rams Chargers': 'Rams,Chargers',
                                  '49ers Raiders': '49ers,Raiders'})
    df['NFL'] = df['NFL'].apply(lambda x: get_team_name(x))
    df['NFL'] = df['NFL'].str.split(",")
    df = df.explode('NFL')
    
    # set index to team
    df.set_index('NFL', inplace=True)
    
    df = df[['Metropolitan area', 'Population (2016 est.)[8]']]
    
    df['Population (2016 est.)[8]'] = df['Population (2016 est.)[8]'].astype(int)
    
    return df



def nfl_correlation(): 
    nfl_df = clean_nfl()
    cities_nfl = clean_cities_nfl()
    
    merged_df = pd.merge(nfl_df, cities_nfl, how='left', left_index=True, right_index=True)
    
    population_by_region = cities_nfl.drop_duplicates('Metropolitan area').sort_values('Metropolitan area')['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df.groupby('Metropolitan area').agg({'W-L%': np.nanmean})['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr, _ = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [59]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def sports_team_performance():
    mlb = clean_mlb()
    mlb_cities = clean_cities_mlb()
    mlb_merged = pd.merge(mlb, mlb_cities, how="left", left_index=True, right_index=True)
    mlb_performance = mlb_merged.groupby('Metropolitan area').agg({'W-L%': np.nanmean})
    
    nhl = clean_nhl()
    nhl_cities = clean_cities_nhl()
    nhl_merged = pd.merge(nhl, nhl_cities, how="left", left_index=True, right_index=True)
    nhl_performance = nhl_merged.groupby('Metropolitan area').agg({'W/L Ratio': np.nanmean})
    nhl_performance.rename(columns={'W/L Ratio': 'W-L%'}, inplace=True)
    
    nba = clean_nba()
    nba_cities = clean_cities_nba()
    nba_merged = pd.merge(nba, nba_cities, how="left", left_index=True, right_index=True)
    nba_performance = nba_merged.groupby('Metropolitan area').agg({'W/L%': np.nanmean})
    nba_performance.rename(columns={'W/L%': 'W-L%'}, inplace=True)
    
    nfl = clean_nfl()
    nfl_cities = clean_cities_nfl()
    nfl_merged = pd.merge(nfl, nfl_cities, how="left", left_index=True, right_index=True)
    nfl_performance = nfl_merged.groupby('Metropolitan area').agg({'W-L%': np.nanmean})
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    all_sports = {'NFL': nfl_performance,
                  'NBA': nba_performance,
                  'NHL': nhl_performance,
                  'MLB': mlb_performance}
    
    p_values = pd.DataFrame(columns=all_sports.keys(), index=all_sports.keys())
    
    for sport1, sport1_df in all_sports.items():
        for sport2, sport2_df in all_sports.items():
            # create new df with common cities
            df = pd.merge(sport1_df, sport2_df, how="inner", left_index=True, right_index=True, suffixes=("_1", "_2"))
            
            # calculate p-value between sports teams
            p_values.loc[sport1, sport2] = stats.ttest_rel(df['W-L%_1'], df['W-L%_2']).pvalue
            
    p_values = p_values.astype(float)

    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
    

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
